## Importaciones

##### Se importan los módulos y las direcciones a utilizar (debe ejecutarse antes de ejecutar cualquier otra celda de codigo en este notebook)

In [1]:
import pandas as pd
import sys
import os
import re
sys.path.append(os.path.join(os.path.abspath(''), '..'))
from src.constants.directions import *
from src.constants.style import *

## DataSet Lagos 

##### Se define la función transform_coordinates, que transformará las coordenadas en grados a decimales.
##### Luego se abre el archivo con pandas y se pasa a  utilizar:
#####   * Se transforman las coordenadas
#####   * Se seleccionan las columnas que se van a utilizar y se ordenan
#####   * Se reemplazan todos los valores NaN por "Desconocido", como se nos pidió
#####   * Se genera el orden para luego usarlo en selección de dificultad
#####   * Se guarda el archivo .csv

In [16]:
def transform_coordinates(coordenadas):
    """
        Transforms cordinates from degrees to float.
    """
    coordenadas=str(coordenadas).replace('S','').replace('O','').replace('"','')
    coordenadas=re.split("[°' ]",coordenadas)
    coordenadas = list(map(lambda x: int(x),coordenadas))
    primer_decimal=(float(coordenadas[0])+(coordenadas[1]/60)+(coordenadas[2]/3600))
    segundo_decimal=(float(coordenadas[3])+(coordenadas[4]/60)+(coordenadas[5]/3600))
    coordenadas=[primer_decimal,segundo_decimal]
    coordenadas=str(coordenadas).replace('[','').replace(']','').replace('"','')
    return coordenadas

lakes_df = pd.read_csv(OLD_LAKES_DIR, sep=',', encoding='utf-8')
lakes_df['Coordenadas'] = lakes_df['Coordenadas'].apply(transform_coordinates)
lakes_df = lakes_df[['Ubicación', 'Superficie (km²)', 'Profundidad máxima (m)', 'Profundidad media (m)', 'Coordenadas', 'Nombre']]
lakes_df.fillna('Desconocido',inplace=True)
lakes_df.sort_values(by=['Superficie (km²)'], ascending=False, inplace=True)

lakes_df.to_csv(LAKES_DIR,index=False,encoding="utf-8")

## DataSet Spotify

##### Primero se abre el archivo con pandas, para a partir de eso modificarlo:
#####   * Se ordenan y filtran las columnas
#####   * Se cambia el nombre de los encabezados
#####   * Se asigna el tipo de la columna "Top Genre" como str (sino da error)
#####   * Se aplica la transformación pedida para encabezados (los que no están en genre_list van en upper(), los demás con el formato de title())
#####   * Se eliminan los valores NaN
#####   * Se genera el orden para luego usarlo en selección de dificultad
#####   * Se guarda el archivo .csv

In [14]:
genre_list = ["edm","dfw","uk","r&b","lgbtq+"]

spotify_df = pd.read_csv(OLD_SPOTIFY_DIR,encoding="utf-8")

spotify_df = spotify_df[["top genre","artist type","year released","top year","bpm","artist"]]
spotify_df.columns = ["Top Genre","Artist Type","Year Released","Top Year","BPM","Artist"]

spotify_df["Top Genre"] = spotify_df["Top Genre"].astype(str)
spotify_df["Top Genre"] = spotify_df["Top Genre"].apply(lambda x: x.title() if x not in genre_list else x.upper())

spotify_df = spotify_df.dropna()
spotify_df.sort_values(by=["Year Released", "Top Year"], ascending=False, inplace=True)

spotify_df.to_csv(SPOTIFY_DIR,index=False,encoding="utf-8")

## DataSet Fifa

##### Se definen dos funciones:
#####   * reemplazar_potencial, que transforma el potencial de numero al formato solicitado
#####   * actualizar_posicion, que se vale del diccionario nuevas_posiciones (que contiene el par abreviación-completo) para escribir las posiciones en el formato solicitado
##### Luego se abre el dataset con pandas, para modificarlo:
#####   * Se genera el orden para luego usarlo en selección de dificultad
#####   * Se filtran y ordenan las columnas
#####   * Se cambia el formato de las columnas a title()
#####   * Se eliminan los valores NaN
#####   * Se actualizan el potencial y la posicion utilizando las funciones ya descriptas
#####   * Se guarda el csv

In [15]:
def reemplazar_potencial(potencial):
    if (potencial<60):
        return "Regular"
    elif (potencial>=60 and potencial <=79):
        return "Bueno"
    elif (potencial>=80 and potencial <=89):
        return "Muy Bueno"
    elif (potencial>=90):
        return "Sobresaliente"

nuevas_posiciones = {
    'ST':'Delantero Centro', 'CF': 'Mediapunta', 'LW':'Extremo Izquierdo',
    'RW':'Extremo Derecho', 'RM':'Mediocampista Derecho', 'LM':'Mediocampista Izquierdo',
    'CAM': 'Mediocampista Ofensivo', 'CM':'Mediocampista', 'CDM':'Mediocampista Defensivo',
    'RB':'Lateral Derecho', 'LB':'Lateral Izquierdo', 'CB':'Defensor Central', 
    'RWB':'Carrilero Derecho', 'LWB':'Carrilero Izquierdo','GK':'Arquero'
}

def actualizar_posicion(posicion):
    todas_posiciones=posicion.split('|')
    posicion=[nuevas_posiciones[x] for x in todas_posiciones]
    posicion=str(posicion).replace(',',' |').replace("'","").replace('[',"").replace(']',"")
    return posicion

fifa_df = pd.read_csv(OLD_FIFA_DIR, sep=';', encoding='utf-8')
fifa_df.sort_values(by=["potential", "overall"], ascending=False, inplace=True)

fifa_df = fifa_df[["team","nationality","position","age","potential","name"]]
fifa_df.columns = [x.title() for x in fifa_df.columns]

fifa_df = fifa_df.dropna()
fifa_df ['Potential'] = fifa_df["Potential"].apply(reemplazar_potencial)
fifa_df['Position'] = fifa_df["Position"].apply(actualizar_posicion)

fifa_df.to_csv(FIFA_DIR,index=False,encoding="utf-8")